In [1]:
from sklearn.metrics import mean_squared_error

from create_matrix import *


def get_tensor(df, start, stop):
#     start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

    dfc = df.copy()

    df = dfc[energy_cols]

    tensor = df.values.reshape((len(df), 7, stop - start))
    return tensor


# def create_region_df_dfc_static(region, year):
#     df, dfc = create_matrix_single_region(region, year)
#     tensor = get_tensor(df, start, stop)
#     static_region = df[['area', 'total_occupants', 'num_rooms']].copy()
#     static_region['area'] = static_region['area'].div(4000)
#     static_region['total_occupants'] = static_region['total_occupants'].div(8)
#     static_region['num_rooms'] = static_region['num_rooms'].div(8)
#     static_region = static_region.values
#     return df, dfc, tensor, static_region



In [2]:
def create_region_df_dfc_static(region, year, start=1, stop=13):
    df, dfc = create_matrix_single_region(region, year)
    tensor = get_tensor(df, start, stop)
    static_region = df[['area', 'total_occupants', 'num_rooms']].copy()
    static_region['area'] = static_region['area'].div(4000)
    static_region['total_occupants'] = static_region['total_occupants'].div(8)
    static_region['num_rooms'] = static_region['num_rooms'].div(8)
    static_region = static_region.values
    return df, dfc, tensor, static_region

In [4]:
df, dfc,tensor,static_region = create_region_df_dfc_static('SanDiego', 2014, 4, 10)

In [5]:
tensor.shape

(39, 7, 6)

In [10]:
H_store = pickle.load(open(os.path.expanduser('~/git/scalable-nilm/aaai18/predictions/H/transfer/case-2/True/True/Austin-SanDiego-0-10.0.pkl'), 'r'))

In [15]:
np.argmin(H_store['Error'])

1

In [10]:
"""

This module saves the A matrix learnt using P = HAS, for various
combinations of parameters, such as number of iterations,
number of season and home factors, Lambda

It can be run as:

### Template
>>>python save_A_graph_laplacian.py case region

### Example
>>> python save_A_graph_laplacian.py 2 Austin
"""

from common import create_region_df_dfc_static
from create_matrix import *
from tensor_custom_core_all import *
import datetime
from common import compute_rmse_fraction, contri, get_tensor, create_region_df_dfc_static

appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}
APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
year = 2014

# case, source, constant_use, start, stop = sys.argv[1:]
case = 2
source = 'Austin'
constant_use = True
start = 5
stop = 11
case = int(case)
start = int(start)
stop = int(stop)
source_df, source_dfc, source_tensor, source_static = create_region_df_dfc_static(source, year, start, stop)

# # using cosine similarity to compute L
source_L = get_L(source_static)

# Seasonal constant constraints
if constant_use == 'True':
    T_constant = np.ones(stop-start).reshape(-1,1)
else:
    T_constant = None
# End

pred = {}
n_splits = 10

algo = 'adagrad'
cost = 'l21'

for appliance in APPLIANCES_ORDER:
    pred[appliance] = []
best_params_global = {}
A_store = {}
err_store = {}




In [ ]:
def learn_HAT_adagrad_graph(case, tensor, L, num_home_factors, num_season_factors, num_iter=2000, lr=0.01, dis=False,
                            lam=1, random_seed=0, eps=1e-8, A_known = None, T_known = None):
    np.random.seed(random_seed)
    cost = cost_graph_laplacian
    # if A_known is not None:
    #     # Don't need to learn A
    #     args_num = [0, 2]
    # else:
    #     args_num = [0, 1, 2]
    args_num=[0,1,2]
    mg = multigrad(cost, argnums=args_num)

    params = {}
    params['M'], params['N'], params['O'] = tensor.shape
    params['a'] = num_home_factors
    params['b'] = num_season_factors
    H_dim_chars = list(cases[case]['HA'].split(",")[0].strip())
    H_dim = tuple(params[x] for x in H_dim_chars)
    A_dim_chars = list(cases[case]['HA'].split(",")[1].split("-")[0].strip())
    A_dim = tuple(params[x] for x in A_dim_chars)
    T_dim_chars = list(cases[case]['HAT'].split(",")[1].split("-")[0].strip())
    T_dim = tuple(params[x] for x in T_dim_chars)

    H = np.random.rand(*H_dim)
    
    # if A_known is not None:
    #     A = A_known
    # else:
    #     A = np.random.rand(*A_dim)
    #     sum_square_gradients_A = np.zeros_like(A)
    A = np.random.rand(*A_dim)
    if A_known is not None:
        A = set_known(A, A_known)
    sum_square_gradients_A = np.zeros_like(A)
    
    
    T = np.random.rand(*T_dim)
    
    sum_square_gradients_H = np.zeros_like(H)
    sum_square_gradients_T = np.zeros_like(T)
    Hs = [H.copy()]
    Ts = [T.copy()]
    As = [A.copy()]
    costs = [cost(H, A, T, L, tensor, lam, case)]
    HATs = [multiply_case(H, A, T, case)]

    

    # GD procedure
    for i in range(num_iter):
        # if A_known is not None:
        #     del_h,  del_t = mg(H, A, T, L, tensor, lam, case)
        # else:
        #     del_h, del_a, del_t = mg(H, A, T, L, tensor, lam, case)
        #     sum_square_gradients_A += eps + np.square(del_a)
        #     lr_a = np.divide(lr, np.sqrt(sum_square_gradients_A))
        #     A -= lr_a * del_a
        del_h, del_a, del_t = mg(H, A, T, L, tensor, lam, case)
        sum_square_gradients_A += eps + np.square(del_a)
        lr_a = np.divide(lr, np.sqrt(sum_square_gradients_A))
        A -= lr_a * del_a
        
        sum_square_gradients_H += eps + np.square(del_h)
        sum_square_gradients_T += eps + np.square(del_t)

        lr_h = np.divide(lr, np.sqrt(sum_square_gradients_H))
        lr_t = np.divide(lr, np.sqrt(sum_square_gradients_T))

        H -= lr_h * del_h
        T -= lr_t * del_t

        if T_known is not None:
            T = set_known(T, T_known)
        if A_known is not None:
            A = set_known(A, A_known)

        # Projection to non-negative space
        H[H < 0] = 1e-8
        A[A < 0] = 1e-8
        T[T < 0] = 1e-8

        As.append(A.copy())
        Ts.append(T.copy())
        Hs.append(H.copy())

        costs.append(cost(H, A, T, L, tensor, lam, case))

        HATs.append(multiply_case(H, A, T, case))
        if i % 500 == 0:
            if dis:
                print(cost(H, A, T, L, tensor, lam, case))
    return H, A, T, Hs, As, Ts, HATs, costs


In [19]:
max_num_iterations = 1300
for learning_rate_cv in [0.1]:
    A_store[learning_rate_cv] = {}
    err_store[learning_rate_cv] = {}
    for num_season_factors_cv in range(2, 3):

        A_store[learning_rate_cv][num_season_factors_cv] = {}
        err_store[learning_rate_cv][num_season_factors_cv] = {}
        for num_home_factors_cv in range(2, 3):
            if case == 4:
                if num_home_factors_cv != num_season_factors_cv:
                    print("Case 4 needs equal # dimensions. Skipping")
                    continue
            A_store[learning_rate_cv][num_season_factors_cv][num_home_factors_cv] = {}
            err_store[learning_rate_cv][num_season_factors_cv][num_home_factors_cv] = {}
            for lam_cv in [0.001]:
                print("-"*80)
                print(datetime.datetime.now())
                print("-" * 80)
                sys.stdout.flush()
                A_store[learning_rate_cv][num_season_factors_cv][num_home_factors_cv][lam_cv] = {}
                err_store[learning_rate_cv][num_season_factors_cv][num_home_factors_cv][lam_cv] = {}
                
                print source_tensor.shape
                H_source, A_source, T_source, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, source_tensor,
                                                                                                source_L,
                                                                                                num_home_factors_cv,
                                                                                                num_season_factors_cv,
                                                                                                num_iter=max_num_iterations,
                                                                                                lr=learning_rate_cv, dis=False,
                                                                                                lam=lam_cv,
                                                                                                T_known=T_constant)
                
                
                print T_source
                for num_iterations in range(100, 1400, 200):
                    A_store[learning_rate_cv][num_season_factors_cv][num_home_factors_cv][lam_cv][num_iterations] = As[num_iterations]
                    print(learning_rate_cv, num_season_factors_cv, num_home_factors_cv, lam_cv, num_iterations, costs[num_iterations])
                    sys.stdout.flush()

                err = {}
                for num_iterations in range(100, 1400, 200):
                    err[num_iterations] = {}
                    for appliance in APPLIANCES_ORDER:
                        pred[appliance] = []
                    for appliance in APPLIANCES_ORDER:
                        pred[appliance].append(pd.DataFrame(HATs[num_iterations][:, appliance_index[appliance], :], index=source_df.index))
                    for appliance in APPLIANCES_ORDER:
                        pred[appliance] = pd.DataFrame(pd.concat(pred[appliance]))

                    for appliance in APPLIANCES_ORDER[1:]:
                        prediction = pred[appliance]
                        if appliance == "hvac":
                            prediction = prediction[range(5-start, 11-start)]
                        err[num_iterations][appliance] = \
                            compute_rmse_fraction(appliance, prediction, source, start, stop)[2]
                    # print("Computed for: {}".format())

                    err_weight = {}
                    for appliance in APPLIANCES_ORDER[1:]:
                        err_weight[appliance] = err[num_iterations][appliance]*contri[source][appliance]
                    mean_err = pd.Series(err_weight).sum()
                    err_store[learning_rate_cv][num_season_factors_cv][num_home_factors_cv][lam_cv][num_iterations] = mean_err

# pickle.dump(A_store, open('../predictions/case-{}-graph_{}_{}_{}_{}_As.pkl'.format(case, source, constant_use, start, stop), 'w'))
# pickle.dump(err_store, open('../predictions/case-{}-graph_{}_{}_{}_{}_errs.pkl'.format(case, source, constant_use, start, stop), 'w'))


--------------------------------------------------------------------------------
2017-09-06 13:12:02.454425
--------------------------------------------------------------------------------
(533, 7, 6)
[[ 5.22935724  4.85548461]
 [ 6.80357525  7.61464996]
 [ 8.34935916  8.24746994]
 [ 8.84420421  9.45900976]
 [ 6.81484295  7.192159  ]
 [ 5.91437255  4.81921679]]
(0.1, 2, 2, 0.001, 100, 541.81357583517922)
(0.1, 2, 2, 0.001, 300, 346.00035599544066)
(0.1, 2, 2, 0.001, 500, 221.38237646713816)
(0.1, 2, 2, 0.001, 700, 183.16802620818515)
(0.1, 2, 2, 0.001, 900, 172.04770698612413)
(0.1, 2, 2, 0.001, 1100, 166.79895521698737)
(0.1, 2, 2, 0.001, 1300, 163.57769186177509)
